# Analyst Workload Analysis

Este notebook demuestra el uso del sistema de KPI de carga laboral.

**Problema a resolver:** Cómo medir la carga de trabajo real de analistas cuando los tickets tienen complejidades muy diferentes.

**Solución:** Sistema de ponderación basado en número de SKUs por transacción.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.kpis.analyst_workload import AnalystWorkloadKPI

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

## 1. Cargar Datos Sintéticos

In [ ]:
# Cargar datasets
orders_df = pd.read_csv('../data/synthetic/purchase_orders.csv')
analysts_df = pd.read_csv('../data/synthetic/analysts.csv')
skus_df = pd.read_csv('../data/synthetic/skus.csv')
order_lines_df = pd.read_csv('../data/synthetic/order_lines.csv')

print(f"Órdenes cargadas: {len(orders_df):,}")
print(f"Analistas: {len(analysts_df)}")
print(f"SKUs en catálogo: {len(skus_df):,}")

## 2. Inicializar Sistema de KPI

In [ ]:
# Crear instancia del sistema
workload_system = AnalystWorkloadKPI(orders_df)

# Ver distribución de complejidad
complexity_dist = workload_system.get_complexity_distribution()
print("\nDISTRIBUCIÓN DE COMPLEJIDAD DE ÓRDENES:")
print("=" * 80)
print(complexity_dist)

## 3. Calcular Carga Laboral por Analista

In [ ]:
# Calcular métricas de carga
workload_by_analyst = workload_system.calculate_workload_by_analyst()

# Join con nombres de analistas
workload_with_names = workload_by_analyst.join(
    analysts_df.set_index('analyst_id')[['analyst_name']], 
    how='left'
)

print("\nCARGA LABORAL POR ANALISTA:")
print("=" * 80)
print(workload_with_names[['analyst_name', 'total_tickets', 'total_skus', 
                            'weighted_workload', 'avg_complexity_weight']])

## 4. Visualizar Distribución de Carga

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Comparación tickets vs carga ponderada
workload_with_names[['total_tickets', 'weighted_workload']].plot(
    kind='bar', 
    ax=axes[0],
    color=['#3498db', '#e74c3c']
)
axes[0].set_title('Tickets vs Carga Ponderada', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Analista')
axes[0].set_ylabel('Cantidad')
axes[0].legend(['Total Tickets', 'Carga Ponderada'])
axes[0].tick_params(axis='x', rotation=45)

# Gráfico 2: Distribución por complejidad
complexity_cols = ['Muy Simple', 'Simple', 'Moderado', 'Complejo']
workload_with_names[complexity_cols].plot(
    kind='bar', 
    stacked=True,
    ax=axes[1],
    color=['#2ecc71', '#f39c12', '#e67e22', '#c0392b']
)
axes[1].set_title('Distribución por Complejidad', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Analista')
axes[1].set_ylabel('Cantidad de Tickets')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Detectar Desbalance de Carga

In [ ]:
# Análisis de balance
imbalance = workload_system.detect_workload_imbalance()

print("\nANÁLISIS DE BALANCE DE CARGA:")
print("=" * 80)
print(f"Carga máxima: {imbalance['max_workload']:.2f}")
print(f"Carga mínima: {imbalance['min_workload']:.2f}")
print(f"Carga promedio: {imbalance['avg_workload']:.2f}")
print(f"Ratio de desbalance: {imbalance['imbalance_ratio']:.2f}x")
print(f"Coeficiente de variación: {imbalance['coefficient_variation']:.1f}%")
print()

if imbalance['coefficient_variation'] > 30:
    print("⚠️  ALERTA: Desbalance significativo detectado (CV > 30%)")
    print("   Recomendación: Revisar distribución de asignaciones")
else:
    print("✓ Distribución de carga relativamente balanceada")

## 6. Identificar Especialización

In [ ]:
# Análisis de especialización
specialization = workload_system.identify_specialization(skus_df, order_lines_df)

# Join con nombres
spec_with_names = specialization.join(
    analysts_df.set_index('analyst_id')[['analyst_name']], 
    how='left'
)

print("\nESPECIALIZACIÓN POR ANALISTA:")
print("=" * 80)
print(spec_with_names[['analyst_name', 'primary_category', 
                       'primary_category_pct', 'categories_handled']])
print()

# Identificar especialistas (>60% en una categoría)
specialists = spec_with_names[spec_with_names['primary_category_pct'] > 60]
if not specialists.empty:
    print(f"\n🎯 Especialistas identificados ({len(specialists)}):")
    for idx, row in specialists.iterrows():
        print(f"   - {row['analyst_name']}: {row['primary_category']} ({row['primary_category_pct']:.1f}%)")

## 7. Generar Reporte Completo

In [ ]:
# Generar reporte textual
report = workload_system.generate_report(analysts_df)
print(report)

## Conclusiones

Este análisis demuestra cómo un sistema de KPI basado en complejidad proporciona insights que un simple conteo de tickets no revelaría:

1. **Carga Real vs Aparente**: Algunos analistas procesan menos tickets pero con mayor complejidad
2. **Especialización Natural**: Se identifican patrones de especialización por categoría
3. **Balance de Carga**: Detectamos desbalances que pueden corregirse

**Aplicabilidad:** Esta misma metodología funciona para:
- Call centers (complejidad por pasos de resolución)
- Soporte técnico (complejidad por sistemas involucrados)
- Análisis financiero (complejidad por cuentas/registros)
- Logística (complejidad por ubicaciones/SKUs)